In [55]:
# current submission is .78
# Things I'd like to do:
    # Hyper Parameters for Linear Regression and Random Forest?
    # Can we use the cabin column in a meaningful way for rows that have that?
    # find other ways to handle NaN Age?
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder, minmax_scale
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#!kaggle competitions download -c titanic
#!kaggle datasets download -d zynicide/wine-reviews # Use this to get data if you want it.

In [3]:
og_df = pd.read_csv('./data/train.csv', engine='python')
og_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
# just rounds the models prediction to 0 or 1.
def get_model_val(df_row, model):
    if(model.predict([df_row])[0] > .5):
        return 1
    else:
        return 0

def clean_data(df):
    # Transformations here.
    new_df = df.copy() 
    new_df = new_df.drop(columns=['Name', 'Ticket', 'Cabin'])

    new_df['Age'] = new_df['Age'].fillna(new_df.Age.median())

    new_df = pd.get_dummies(new_df, columns=['Sex', 'Embarked', 'Pclass'])
    new_df = pd.DataFrame(minmax_scale(new_df), columns=new_df.columns)

    return new_df

def test_data(test_df, ans_df):
    # For local testing so I don't have to submit all the time.
    correct = 0
    total = 0 
    for index, row in test_df.iterrows():
        if row['PassengerId'] in ans_df['PassengerId'].values:
            if ans_df.loc()[ans_df['PassengerId'] == row['PassengerId']].iloc[0]['Survived'] == row['Survived']:
                correct = correct +1
        total += 1 
    return "{} out of {} for a % of {}".format(correct, total, correct/total)

In [5]:
df = clean_data(og_df)
df_passenger_id = og_df['PassengerId'].copy()
df_labels = df['Survived'].copy()
corr_matrix = df.corr()
df = df.drop(columns=['Survived', 'PassengerId'])

In [6]:
corr_matrix['Survived'].sort_values(ascending=False)

Survived       1.000000
Sex_female     0.543351
Pclass_1       0.285904
Fare           0.257307
Embarked_C     0.168240
Pclass_2       0.093349
Parch          0.081629
Embarked_Q     0.003650
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.064910
Embarked_S    -0.155660
Pclass_3      -0.322308
Sex_male      -0.543351
Name: Survived, dtype: float64

In [7]:
lin_reg = LinearRegression()
lin_reg.fit(df,df_labels)
model = lin_reg
lin_reg.predict([df.iloc[1]])[0] # sanity check

0.9230305324741012

In [112]:
log_reg = LogisticRegression(C=.01)
log_reg.fit(df,df_labels)
model = log_reg
log_reg.predict([df.iloc[1]])[0] # sanity check

1.0

In [51]:
rid_reg = Ridge()
rid_reg.fit(df,df_labels)
model = rid_reg
rid_reg.predict([df.iloc[1]])[0] # sanity check

0.9270699631020117

In [47]:
las_reg = Lasso(alpha=.001, max_iter=100)
las_reg.fit(df,df_labels)
model = las_reg
las_reg.predict([df.iloc[1]])[0] # sanity check

0.924629616082303

In [108]:
tree_reg = DecisionTreeRegressor(max_depth=6)
tree_reg.fit(df, df_labels)
model = tree_reg
tree_reg.predict([df.iloc[1]])[0] # sanity check

1.0

In [ ]:
forest_reg = RandomForestRegressor()
forest_reg.fit(df, df_labels)
model = forest_reg
forest_reg.predict([df.iloc[1]])[0] # sanity check

In [14]:
knn = KNeighborsClassifier(n_neighbors=2) # can play with this a bit.
knn.fit(df,df_labels)
model = knn
knn.predict([df.iloc[1]])[0] # sanity check

1.0

In [113]:
tr_og_df = pd.read_csv('./data/test.csv', engine='python')
tr_df_passengerId = tr_og_df['PassengerId'].copy()
tr_df = clean_data(tr_og_df)
tr_df['Fare'] = tr_df['Fare'].fillna(tr_df.Fare.median())
tr_df = tr_df.drop(columns=['PassengerId'])
tr_df['Survived'] = tr_df.apply(lambda row: get_model_val(row, model), axis=1)
final_df = pd.DataFrame(list(zip(tr_df_passengerId, tr_df['Survived'])),columns=['PassengerId', 'Survived'])

In [116]:
final_df.to_csv('output.csv', index=False)
#!kaggle competitions submit -c titanic -f ./output.csv

In [115]:
ans_df = pd.read_csv('./data/answers.csv', engine='python')
ans_df = ans_df.rename({1: 'PassengerId', 2:'Survived'}, axis='columns')

test_df = tr_og_df.merge(ans_df, left_on="Name", right_on="name")
test_df_only_survived = test_df[['survived', 'PassengerId']].copy()
test_df_only = test_df_only_survived.rename({'survived': 'Survived'}, axis='columns')
ans_df = test_df_only
ans_df = ans_df.drop(1) # Something weird happens and we have to drop this one.
print(test_data(final_df, ans_df))
#ans_df = ans_df[ans_df['name'] == tr_og_df['Name']]

#ans_df.drop(ans_df.index[0:892], inplace=True) # We can't do this as test is not sorted.
#final_df = final_df.drop([417], axis="index") # We have to do this before testing for some reason.

308 out of 418 for a % of 0.7368421052631579
